In [1]:
%load_ext autoreload
%autoreload 2

## The health and fitness instructor!
1. Take input: Weight and height
2. Goals and current challenges
3. Favorite meals
4. Workout planner
5. Nutrition planner

In [ ]:
from crewai import Crew, Process, Agent, Task
from crewai.tools import tool